In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
import os
from sklearn.cluster import MiniBatchKMeans
from sklearn.cluster import KMeans
from sklearn import metrics
import scipy.sparse


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/
%ls *.csv

/content/drive/MyDrive
wiki_movie_plots_deduped.csv


In [ ]:
dir_file = os.getcwd() # returns path to current directory
files_dir = os.listdir(dir_file)  # list of files in current directory

csv_files = [f for f in files_dir if f.endswith('csv')]
print(csv_files)
movie_file = csv_files[0]

movie_df = pd.read_csv(movie_file)
print(movie_df.columns)  # the columns
print(movie_df.shape)
movie_df.head()

['wiki_movie_plots_deduped.csv']
Index(['Release Year', 'Title', 'Origin/Ethnicity', 'Director', 'Cast',
       'Genre', 'Wiki Page', 'Plot'],
      dtype='object')
(34886, 8)


,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Wiki Page,Plot
0,1901,Kansas Saloon Smashers,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Kansas_Saloon_Sm...,"A bartender is working at a saloon, serving dr..."
1,1901,Love by the Light of the Moon,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Love_by_the_Ligh...,"The moon, painted with a smiling face hangs ov..."
2,1901,The Martyred Presidents,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/The_Martyred_Pre...,"The film, just over a minute long, is composed..."
3,1901,"Terrible Teddy, the Grizzly King",American,Unknown,NaN,unknown,"https://en.wikipedia.org/wiki/Terrible_Teddy,_...",Lasting just 61 seconds and consisting of two ...
4,1902,Jack and the Beanstalk,American,"George S. Fleming, Edwin S. Porter",NaN,unknown,https://en.wikipedia.org/wiki/Jack_and_the_Bea...,The earliest known adaptation of the classic f...


In [ ]:
# Get the top 35 genres
top_genres = movie_df['Genre'].value_counts().head(11).index.tolist()[1:]
# Display the array
print(top_genres)

['drama', 'comedy', 'horror', 'action', 'thriller', 'romance', 'western', 'crime', 'adventure', 'musical']


In [ ]:
# Remove rows where 'Genre' is labeled as 'unknown'
data_filtered = movie_df[['Plot', 'Genre']][movie_df['Genre'].isin(top_genres)]
data_filtered.head()

# Confirm the removal by checking the counts of 'unknown' in the Genre column
filtered_unknown_count = data_filtered['Genre'].value_counts().get('unknown', 0)
print("Count of 'unknown' in filtered data:", filtered_unknown_count)


Count of 'unknown' in filtered data: 0


#Write a Naiive Bayes Classifier
#Use Naive Bayes to classify a novel document as some genre

In [ ]:
import numpy as np
import spacy   # another tokenizer, lemmatizer (has --> be)
nlp = spacy.load('en_core_web_sm')
nlp.disable_pipes('parser', 'ner')

['parser', 'ner']

In [ ]:
def nlp_processing(doc): # from indexing.ipyns
    tokens = nlp(doc)

    #print(type(tokens))
    # eliminates stop words  and non alpha num and converts all to lower case
    terms = [token.lemma_.lower() for token in tokens if token.is_alpha and (not token.is_stop)]

    return terms

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

# Assuming 'plot' is your text column and 'genre' or another column is your target
X = data_filtered['Plot']  # Replace 'plot' with your actual text column
y = data_filtered['Genre']  # Replace 'genre' with your target class column

# Convert text to feature vectors
vectorizer = CountVectorizer(tokenizer = nlp_processing, min_df = 30) # ? why 50
X_vectorized = vectorizer.fit_transform(X)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_vectorized, y, test_size=0.2, random_state=42)


In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix

# Define parameter grid for alpha
param_grid = {'alpha': [0.1, 0.5, 1.0, 1.5, 2.0]}

# Instantiate GridSearchCV with MultinomialNB and alpha parameter grid
grid_search = GridSearchCV(MultinomialNB(), param_grid, cv=5, scoring='accuracy')

# Train the model with grid search
grid_search.fit(X_train, y_train)

# Retrieve the best model and its parameters
best_nb_classifier = grid_search.best_estimator_
best_alpha = grid_search.best_params_['alpha']
print(f"Best alpha: {best_alpha}")

# Predict on the test set using the best model
y_pred = best_nb_classifier.predict(X_test)

# Calculate and print accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy with optimized alpha: {accuracy:.2f}')


Best alpha: 0.1
Accuracy with optimized alpha: 0.56


In [ ]:
# Predict on the test set
y_pred = best_nb_classifier.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')
recall = metrics.recall_score(y_test, y_pred, average='weighted')
print(f'Recall: {recall:.2f}')
precision = metrics.precision_score(y_test, y_pred, average='weighted')
print(f'Precision: {precision:.2f}')

# Calculate F1 score
f1 = f1_score(y_test, y_pred, average='weighted')
print(f'F1 Score (weighted): {f1:.2f}')

# Macro and Micro averaged scores
macro_f1 = f1_score(y_test, y_pred, average='macro')
micro_f1 = f1_score(y_test, y_pred, average='micro')
print(f'Macro-averaged F1: {macro_f1:.2f}')
print(f'Micro-averaged F1: {micro_f1:.2f}')

# Confusion Matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("\nConfusion Matrix:")
print(conf_matrix)

Accuracy: 0.56
Recall: 0.56
Precision: 0.58
F1 Score (weighted): 0.56
Macro-averaged F1: 0.51
Micro-averaged F1: 0.56

Confusion Matrix:
[[121   9   5   8  11   4   0  34  10   2]
 [  4  61   6   2  16   1   1   1   1   2]
 [ 28  33 491  17 172  16  38  56  25   6]
 [ 11   1  25  50  21   4   1   0  14   1]
 [ 52  55 183  38 642  20  19 134  41  21]
 [  4   1   9   1  14 191   2   9  16   0]
 [  0   2  30   1  12   0  37   3   0   3]
 [  5   1  15   0  41   1   6 107   1   1]
 [ 17   7  18   6  43  34   0  19  42   0]
 [  1   4   4   3  11   1   2   1   0 145]]


#Use Bert to Classify some novel document as some genre



In [ ]:
import torch
!nvidia-smi

Sun Dec  8 22:49:39 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8               9W /  70W |      3MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.empty_cache()

In [ ]:
torch.cuda.get_device_name(0)

'Tesla T4'

In [ ]:
# 1. Install and upgrade necessary packages (Jupyter Notebook)
#!pip install --upgrade sentence-transformers transformers datasets scikit-learn

# After running the above cell, manually restart the runtime/kernel.

# 2. Import Libraries
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Dataset
from sentence_transformers import SentenceTransformer
import torch
from torch import nn
from sklearn.metrics import accuracy_score, recall_score, precision_score


X = data_filtered['Plot']   # Replace 'Plot' with your actual text column
y = data_filtered['Genre']  # Replace 'Genre' with your target class column

# Encode labels as integers
le = LabelEncoder()
y_encoded = le.fit_transform(y)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(
    X, y_encoded, test_size=0.2, random_state=42
)

# 4. Create InputExamples
class ClassificationDataset(Dataset):
    def __init__(self, texts, labels):
        self.texts = texts.tolist()
        self.labels = labels.tolist()

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        return self.texts[idx], self.labels[idx]

train_dataset = ClassificationDataset(X_train, y_train)
test_dataset = ClassificationDataset(X_test, y_test)

# 5. Create DataLoaders
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=32)
test_dataloader = DataLoader(test_dataset, shuffle=False, batch_size=32)

# 6. Initialize the Model with Classification Head
class ClassificationModel(nn.Module):
    def __init__(self, transformer_model, num_labels):
        super(ClassificationModel, self).__init__()
        self.transformer = transformer_model
        self.classifier = nn.Linear(transformer_model.get_sentence_embedding_dimension(), num_labels)

    def forward(self, input_texts):
        embeddings = self.transformer.encode(input_texts, convert_to_tensor=True, device=device)
        logits = self.classifier(embeddings)
        return logits

# Initialize SentenceTransformer
transformer_model = SentenceTransformer('all-MiniLM-L6-v2')

# Initialize Classification Model
num_classes = len(le.classes_)
model = ClassificationModel(transformer_model, num_classes)

# Move model to GPU if available
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

# 7. Define Loss Function and Optimizer
criterion = nn.NLLLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)

# 8. Training Loop
epochs = 20

for epoch in range(epochs):
    model.train()
    total_loss = 0
    for batch in train_dataloader:
        texts, labels = batch
        labels = torch.tensor(labels).to(device)

        optimizer.zero_grad()
        outputs = model(texts)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(train_dataloader)
    print(f"Epoch {epoch + 1}/{epochs}, Loss: {avg_loss:.4f}")

    # Evaluation
    model.eval()
    preds = []
    true_labels = []
    with torch.no_grad():
        for batch in test_dataloader:
            texts, labels = batch
            labels = labels.to(device)
            outputs = model(texts)
            _, predicted = torch.max(outputs, 1)
            preds.extend(predicted.cpu().numpy())
            true_labels.extend(labels.cpu().numpy())



    # 2. Calculate Metrics
    accuracy = accuracy_score(true_labels, preds)
    recall = recall_score(true_labels, preds, average='weighted')     # Use 'macro' or 'micro' based on your needs
    precision = precision_score(true_labels, preds, average='weighted')  # Use 'macro' or 'micro' based on your needs
    f1 = f1_score(true_labels, preds, average='weighted')

    macro_f1 = f1_score(true_labels, preds, average='macro')
    micro_f1 = f1_score(true_labels, preds, average='micro')

    # 3. Print Metrics
    print(f"Validation Accuracy:  {accuracy:.4f}")
    print(f"Validation Recall:     {recall:.4f}")
    print(f"Validation Precision:  {precision:.4f}")
    print(f"Validation F1 Score:   {f1:.4f}")
    print(f"Macro-Averaged F1:     {macro_f1:.4f}")
    print(f"Micro-Averaged F1:     {micro_f1:.4f}")

    # 4. Confusion Matrix
    conf_matrix = confusion_matrix(true_labels, preds)
    print("\nConfusion Matrix:")
    print(conf_matrix)

# 9. Save the Trained Model
torch.save(model.state_dict(), 'classification_model.pth')


/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

Validation Accuracy:  0.4103
Validation Recall:     0.4103
Validation Precision:  0.4067
Validation F1 Score:   0.3116
Macro-Averaged F1:     0.1371
Micro-Averaged F1:     0.4103

Confusion Matrix:
[[   2    0    4    0  193    0    0    2    0    3]
 [   0    0    4    1   83    1    0    1    0    5]
 [   0    0  234    0  636    0    0    6    0    6]
 [   0    0   18    0  109    0    0    0    0    1]
 [   1    0   77    0 1100    1    0   18    0    8]
 [   0    0   15    0  229    3    0    0    0    0]
 [   0    0   27    0   59    0    0    0    0    2]
 [   0    0    1    0  170    0    0    7    0    0]
 [   0    0   15    0  170    1    0    0    0    0]
 [   0    0   10    0  119    0    0    0    0   43]]
Epoch 19/20, Loss: -1.0111


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
<ipython-input-17-d860726ae668>:83: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(labels).to(device)


Validation Accuracy:  0.4103
Validation Recall:     0.4103
Validation Precision:  0.4066
Validation F1 Score:   0.3118
Macro-Averaged F1:     0.1369
Micro-Averaged F1:     0.4103

Confusion Matrix:
[[   2    0    5    0  192    0    0    2    0    3]
 [   0    0    4    1   83    1    0    1    0    5]
 [   0    0  236    0  635    0    0    6    0    5]
 [   0    0   18    0  109    0    0    0    0    1]
 [   1    0   78    0 1099    1    0   18    0    8]
 [   0    0   15    0  229    3    0    0    0    0]
 [   0    0   27    0   59    0    0    0    0    2]
 [   0    0    1    0  169    0    0    8    0    0]
 [   0    0   15    0  170    1    0    0    0    0]
 [   0    0   13    0  118    0    0    0    0   41]]
Epoch 20/20, Loss: -1.0644


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Validation Accuracy:  0.4112
Validation Recall:     0.4112
Validation Precision:  0.4066
Validation F1 Score:   0.3132
Macro-Averaged F1:     0.1376
Micro-Averaged F1:     0.4112

Confusion Matrix:
[[   2    0    6    0  191    0    0    2    0    3]
 [   0    0    4    1   83    1    0    1    0    5]
 [   0    0  241    0  631    0    0    6    0    4]
 [   0    0   18    0  109    0    0    0    0    1]
 [   1    0   80    0 1097    1    0   18    0    8]
 [   0    0   16    0  228    3    0    0    0    0]
 [   0    0   27    0   59    0    0    0    0    2]
 [   0    0    1    0  169    0    0    8    0    0]
 [   0    0   16    0  169    1    0    0    0    0]
 [   0    0   14    0  117    0    0    0    0   41]]
